# Getting Started with Snowflake Dynamic Tables

This notebook demonstrates how to build declarative data pipelines using Snowflake Dynamic Tables for continuous data transformations, data validation, and alerting. Dynamic Tables automatically refresh based on defined queries and target freshness, simplifying data pipeline management without manual scheduling.

**Original Quickstart:** https://quickstarts.snowflake.com/guide/getting_started_with_dynamic_tables/index.html

## Prerequisites
- **Packages Required:** The Python UDTFs in this demo use the `Faker` package which is available in Snowflake's Anaconda channel. No additional package installation needed.
- **Permissions:** Ensure you have privileges to create databases, schemas, warehouses, tables, and dynamic tables.
- **Change Tracking:** Will be automatically enabled on base tables as needed.


In [ ]:
-- Display current session context and connection information
SELECT 
    CURRENT_DATABASE() as current_database,
    CURRENT_SCHEMA() as current_schema,
    CURRENT_WAREHOUSE() as current_warehouse,
    CURRENT_USER() as current_user,
    CURRENT_ROLE() as current_role,
    CURRENT_REGION() as current_region;


In [ ]:
-- Set up database, schema, and warehouse for the demo
CREATE DATABASE IF NOT EXISTS DEMO;
CREATE SCHEMA IF NOT EXISTS DEMO.DT_DEMO;
USE SCHEMA DEMO.DT_DEMO;

CREATE WAREHOUSE IF NOT EXISTS XSMALL_WH 
    WAREHOUSE_TYPE = STANDARD
    WAREHOUSE_SIZE = XSMALL
    AUTO_SUSPEND = 300
    AUTO_RESUME = TRUE;


## Sample Data Generation

We'll create three source tables using Python UDTFs to generate realistic sample data:
1. **CUST_INFO** - Customer information with spending limits
2. **PROD_STOCK_INV** - Product inventory with stock levels
3. **SALESDATA** - Raw sales transactions in JSON format


In [ ]:
-- Create Python UDTF to generate customer information data
CREATE OR REPLACE FUNCTION gen_cust_info(num_records number)
RETURNS TABLE (custid number(10), cname varchar(100), spendlimit number(10,2))
LANGUAGE PYTHON
RUNTIME_VERSION=3.10
HANDLER='CustTab'
PACKAGES = ('Faker')
AS $$
from faker import Faker
import random

fake = Faker()
# Generate a list of customers  

class CustTab:
    # Generate multiple customer records
    def process(self, num_records):
        customer_id = 1000 # Starting customer ID                 
        for _ in range(num_records):
            custid = customer_id + 1
            cname = fake.name()
            spendlimit = round(random.uniform(1000, 10000),2)
            customer_id += 1
            yield (custid,cname,spendlimit)
$$;

CREATE OR REPLACE TABLE cust_info AS 
SELECT * FROM table(gen_cust_info(1000)) ORDER BY 1;


In [ ]:
-- Create Python UDTF to generate product inventory data
CREATE OR REPLACE FUNCTION gen_prod_inv(num_records number)
RETURNS TABLE (pid number(10), pname varchar(100), stock number(10,2), stockdate date)
LANGUAGE PYTHON
RUNTIME_VERSION=3.10
HANDLER='ProdTab'
PACKAGES = ('Faker')
AS $$
from faker import Faker
import random
from datetime import datetime, timedelta
fake = Faker()

class ProdTab:
    # Generate multiple product records
    def process(self, num_records):
        product_id = 100 # Starting customer ID                 
        for _ in range(num_records):
            pid = product_id + 1
            pname = fake.catch_phrase()
            stock = round(random.uniform(500, 1000),0)
            # Get the current date
            current_date = datetime.now()
            
            # Calculate the maximum date (3 months from now)
            min_date = current_date - timedelta(days=90)
            
            # Generate a random date within the date range
            stockdate = fake.date_between_dates(min_date,current_date)

            product_id += 1
            yield (pid,pname,stock,stockdate)
$$;

CREATE OR REPLACE TABLE prod_stock_inv AS 
SELECT * FROM table(gen_prod_inv(100)) ORDER BY 1;


In [ ]:
-- Create Python UDTF to generate sales transaction data in JSON format
CREATE OR REPLACE FUNCTION gen_cust_purchase(num_records number,ndays number)
RETURNS TABLE (custid number(10), purchase variant)
LANGUAGE PYTHON
RUNTIME_VERSION=3.10
HANDLER='genCustPurchase'
PACKAGES = ('Faker')
AS $$
from faker import Faker
import random
from datetime import datetime, timedelta

fake = Faker()

class genCustPurchase:
    # Generate multiple customer purchase records
    def process(self, num_records,ndays):       
        for _ in range(num_records):
            c_id = fake.random_int(min=1001, max=1999)
            
            customer_purchase = {
                'custid': c_id,
                'purchased': []
            }
            # Get the current date
            current_date = datetime.now()
            
            # Calculate the maximum date (days from now)
            min_date = current_date - timedelta(days=ndays)
            
            # Generate a random date within the date range
            pdate = fake.date_between_dates(min_date,current_date)
            
            purchase = {
                'prodid': fake.random_int(min=101, max=199),
                'quantity': fake.random_int(min=1, max=5),
                'purchase_amount': round(random.uniform(10, 1000),2),
                'purchase_date': pdate
            }
            customer_purchase['purchased'].append(purchase)
            
            yield (c_id,purchase)
$$;

-- Create table and insert records 
CREATE OR REPLACE TABLE salesdata AS 
SELECT * FROM table(gen_cust_purchase(10000,10));


In [ ]:
-- Verify sample data has been created successfully
-- Customer information table, each customer has spending limits
SELECT 'CUST_INFO' as table_name, COUNT(*) as record_count FROM cust_info
UNION ALL
-- Product stock table, each product has stock level from fulfillment day
SELECT 'PROD_STOCK_INV' as table_name, COUNT(*) as record_count FROM prod_stock_inv
UNION ALL
-- Sales data for products purchased online by various customers
SELECT 'SALESDATA' as table_name, COUNT(*) as record_count FROM salesdata;


In [ ]:
-- Preview sample data from customer info table
SELECT * FROM cust_info LIMIT 5;


In [ ]:
-- Preview sample data from product inventory table
SELECT * FROM prod_stock_inv LIMIT 5;


In [ ]:
-- Preview sample data from sales transactions table
SELECT * FROM salesdata LIMIT 5;


## Building the Dynamic Tables Pipeline

Now we'll create a series of Dynamic Tables to build our data pipeline. The pipeline will:
1. Extract and transform sales data from JSON format
2. Join with customer and product information 
3. Create sales reports with SCD Type 2 transformations
4. Build cumulative totals using Python UDTFs
5. Create data validation and alerting


In [ ]:
-- Create first Dynamic Table: Extract sales data and join with customer info
USE SCHEMA DEMO.DT_DEMO;

CREATE OR REPLACE DYNAMIC TABLE customer_sales_data_history
    LAG='DOWNSTREAM'
    WAREHOUSE=XSMALL_WH
AS
SELECT 
    s.custid as customer_id,
    c.cname as customer_name,
    s.purchase:"prodid"::number(5) as product_id,
    s.purchase:"purchase_amount"::number(10) as saleprice,
    s.purchase:"quantity"::number(5) as quantity,
    s.purchase:"purchase_date"::date as salesdate
FROM
    cust_info c INNER JOIN salesdata s ON c.custid = s.custid;


In [ ]:
-- Verify the first Dynamic Table and check record count
SELECT * FROM customer_sales_data_history LIMIT 10;
SELECT COUNT(*) as total_records FROM customer_sales_data_history;


In [ ]:
-- Create second Dynamic Table: Sales report with SCD Type 2 transformation
CREATE OR REPLACE DYNAMIC TABLE salesreport
    LAG = '1 MINUTE'
    WAREHOUSE=XSMALL_WH
AS
    SELECT
        t1.customer_id,
        t1.customer_name, 
        t1.product_id,
        p.pname as product_name,
        t1.saleprice,
        t1.quantity,
        (t1.saleprice/t1.quantity) as unitsalesprice,
        t1.salesdate as CreationTime,
        customer_id || '-' || t1.product_id  || '-' || t1.salesdate AS CUSTOMER_SK,
        LEAD(CreationTime) OVER (PARTITION BY t1.customer_id ORDER BY CreationTime ASC) AS END_TIME
    FROM 
        customer_sales_data_history t1 INNER JOIN prod_stock_inv p 
        ON t1.product_id = p.pid;


In [ ]:
-- Verify the sales report Dynamic Table
SELECT * FROM salesreport LIMIT 10;
SELECT COUNT(*) as total_records FROM salesreport;


In [ ]:
-- Test the Dynamic Tables pipeline by adding new data
INSERT INTO salesdata SELECT * FROM table(gen_cust_purchase(10000,2));

-- Check raw base table
SELECT COUNT(*) as salesdata_count FROM salesdata;


In [ ]:
-- Check Dynamic Tables after refresh (wait a minute for automatic refresh)
SELECT COUNT(*) as customer_sales_history_count FROM customer_sales_data_history;


## Query Types Showcase: Basic Incremental Operations

Dynamic Tables support different types of queries that are optimized for incremental refresh mode. These operations can efficiently process only the changed data rather than requiring a full table refresh. Let's demonstrate the four basic incremental operation types:

1. **Simple Aggregations** - Basic SUM, COUNT, AVG operations on streaming data
2. **Filter Operations** - WHERE clauses filtering on timestamp or ID columns  
3. **Simple Joins** - INNER and LEFT JOINs between tables with proper join keys
4. **UNION Operations** - Combining similar datasets with UNION ALL

Each example will create a Dynamic Table that showcases these incremental-friendly patterns.


In [ ]:
-- Example 1: Simple Aggregations Dynamic Table
-- Demonstrates basic SUM, COUNT, AVG operations optimal for incremental refresh
CREATE OR REPLACE DYNAMIC TABLE daily_sales_summary
    LAG = '1 MINUTE'
    WAREHOUSE = XSMALL_WH
AS
SELECT 
    DATE(sr.creationtime) as sales_date,
    COUNT(*) as total_transactions,
    SUM(sr.saleprice) as total_revenue,
    AVG(sr.saleprice) as avg_transaction_amount,
    SUM(sr.quantity) as total_units_sold,
    COUNT(DISTINCT sr.customer_id) as unique_customers
FROM salesreport sr
GROUP BY DATE(sr.creationtime);


In [ ]:
-- Example 2: Filter Operations Dynamic Table  
-- Demonstrates WHERE clauses filtering on timestamp/ID columns for incremental processing
CREATE OR REPLACE DYNAMIC TABLE high_value_recent_sales
    LAG = '1 MINUTE'
    WAREHOUSE = XSMALL_WH
AS
SELECT 
    sr.customer_id,
    sr.customer_name,
    sr.product_id,
    sr.product_name,
    sr.saleprice,
    sr.quantity,
    sr.creationtime
FROM salesreport sr
WHERE 
    sr.saleprice > 500  -- Filter for high-value transactions
    AND sr.creationtime >= DATEADD('day', -7, CURRENT_DATE())  -- Recent transactions only
    AND sr.customer_id BETWEEN 1500 AND 1800;  -- Specific customer segment


In [ ]:
-- Alternative Option 2: Rolling 7-day window using a separate staging table
-- This approach creates a "control table" to manage the date window
-- First create a control table for date management:
-- CREATE TABLE date_control (cutoff_date DATE);
-- INSERT INTO date_control VALUES (DATEADD('day', -7, CURRENT_DATE()));

-- Then create the dynamic table with static reference:
-- CREATE OR REPLACE DYNAMIC TABLE high_value_recent_sales_rolling
--     LAG = '1 MINUTE'
--     WAREHOUSE = XSMALL_WH
-- AS
-- SELECT 
--     sr.customer_id,
--     sr.customer_name,
--     sr.product_id,
--     sr.product_name,
--     sr.saleprice,
--     sr.quantity,
--     sr.creationtime
-- FROM salesreport sr
-- CROSS JOIN date_control dc
-- WHERE 
--     sr.saleprice > 500  
--     AND sr.creationtime >= dc.cutoff_date  -- Reference to control table
--     AND sr.customer_id BETWEEN 1500 AND 1800;

-- To update the window, simply update the control table:
-- UPDATE date_control SET cutoff_date = DATEADD('day', -7, CURRENT_DATE());


In [ ]:
-- Example 3: Simple Joins Dynamic Table
-- Demonstrates INNER and LEFT JOINs with proper join keys for incremental processing  
CREATE OR REPLACE DYNAMIC TABLE customer_purchase_analysis
    LAG = '1 MINUTE'
    WAREHOUSE = XSMALL_WH
AS
SELECT 
    c.custid,
    c.cname as customer_name,
    c.spendlimit,
    sr.saleprice,
    sr.quantity,
    sr.creationtime,
    -- Calculate spending as percentage of limit
    ROUND((sr.saleprice / c.spendlimit) * 100, 2) as spend_percentage,
    -- Flag customers approaching their limit
    CASE 
        WHEN (sr.saleprice / c.spendlimit) > 0.8 THEN 'High Risk'
        WHEN (sr.saleprice / c.spendlimit) > 0.6 THEN 'Medium Risk'  
        ELSE 'Low Risk'
    END as risk_category
FROM cust_info c
INNER JOIN salesreport sr ON c.custid = sr.customer_id;


In [ ]:
-- Example 4: UNION Operations Dynamic Table
-- Demonstrates combining similar datasets with UNION ALL for incremental processing
CREATE OR REPLACE DYNAMIC TABLE unified_transaction_log
    LAG = '1 MINUTE'
    WAREHOUSE = XSMALL_WH
AS
-- High value transactions (>= 750)
SELECT 
    'HIGH_VALUE' as transaction_category,
    customer_id,
    customer_name,
    product_id,
    saleprice,
    quantity,
    creationtime,
    'Premium Customer' as customer_tier
FROM salesreport 
WHERE saleprice >= 750

UNION ALL

-- Medium value transactions (250-749)
SELECT 
    'MEDIUM_VALUE' as transaction_category,
    customer_id,
    customer_name,
    product_id,
    saleprice,
    quantity,
    creationtime,
    'Standard Customer' as customer_tier
FROM salesreport 
WHERE saleprice >= 250 AND saleprice < 750

UNION ALL

-- Low value transactions (< 250)
SELECT 
    'LOW_VALUE' as transaction_category,
    customer_id,
    customer_name,
    product_id,
    saleprice,
    quantity,
    creationtime,
    'Basic Customer' as customer_tier
FROM salesreport 
WHERE saleprice < 250;


In [ ]:
-- Verify the new Dynamic Tables were created successfully
SELECT 'DAILY_SALES_SUMMARY' as table_name, COUNT(*) as record_count FROM daily_sales_summary
UNION ALL
SELECT 'HIGH_VALUE_RECENT_SALES' as table_name, COUNT(*) as record_count FROM high_value_recent_sales
UNION ALL
SELECT 'CUSTOMER_PURCHASE_ANALYSIS' as table_name, COUNT(*) as record_count FROM customer_purchase_analysis
UNION ALL
SELECT 'UNIFIED_TRANSACTION_LOG' as table_name, COUNT(*) as record_count FROM unified_transaction_log;


### Testing Incremental Refresh Behavior

Now let's test the incremental refresh behavior of our Dynamic Tables by:
1. Adding new sales data to trigger automatic refreshes
2. Monitoring the refresh history to verify incremental processing
3. Comparing before/after record counts to confirm updates


In [ ]:
-- Insert new sales data to trigger Dynamic Table refreshes
-- This will demonstrate incremental processing capabilities
INSERT INTO salesdata SELECT * FROM table(gen_cust_purchase(5000, 3));

-- Check the updated base table count
SELECT 'SALESDATA_AFTER_INSERT' as table_name, COUNT(*) as record_count FROM salesdata;


In [ ]:
-- Wait a moment for Dynamic Tables to refresh, then check updated counts
-- The LAG='1 MINUTE' setting means tables should refresh within a minute
SELECT 'DAILY_SALES_SUMMARY_AFTER' as table_name, COUNT(*) as record_count FROM daily_sales_summary
UNION ALL
SELECT 'HIGH_VALUE_RECENT_SALES_AFTER' as table_name, COUNT(*) as record_count FROM high_value_recent_sales
UNION ALL
SELECT 'CUSTOMER_PURCHASE_ANALYSIS_AFTER' as table_name, COUNT(*) as record_count FROM customer_purchase_analysis
UNION ALL
SELECT 'UNIFIED_TRANSACTION_LOG_AFTER' as table_name, COUNT(*) as record_count FROM unified_transaction_log;


In [ ]:
-- Check Dynamic Table refresh history to verify incremental vs full refresh behavior
-- REFRESH_TYPE indicates whether the refresh was 'INCREMENTAL' or 'FULL'
-- After fixing high_value_recent_sales (removed CURRENT_DATE()), it should now show INCREMENTAL refreshes
SELECT 
    NAME as dynamic_table_name,
    REFRESH_TYPE,
    REFRESH_REASON,
    DATA_TIMESTAMP,
    REFRESH_START_TIME,
    REFRESH_END_TIME,
    CREDITS_USED,
    BYTES_SCANNED,
    ROWS_PRODUCED
FROM 
    TABLE(INFORMATION_SCHEMA.DYNAMIC_TABLE_REFRESH_HISTORY())
WHERE 
    NAME IN ('DAILY_SALES_SUMMARY', 'HIGH_VALUE_RECENT_SALES', 'CUSTOMER_PURCHASE_ANALYSIS', 'UNIFIED_TRANSACTION_LOG')
    AND REFRESH_START_TIME >= DATEADD('hour', -1, CURRENT_TIMESTAMP())
ORDER BY 
    NAME, DATA_TIMESTAMP DESC, REFRESH_END_TIME DESC;


### Query Types Summary

The Dynamic Tables we just created demonstrate the four basic incremental operation patterns:

1. **`daily_sales_summary`** - Simple aggregations (SUM, COUNT, AVG) that can efficiently process only new data
2. **`high_value_recent_sales`** - Filter operations using WHERE clauses with static dates and ID columns (fixed to use static date instead of CURRENT_DATE() for incremental processing)  
3. **`customer_purchase_analysis`** - Simple joins between tables using proper join keys
4. **`unified_transaction_log`** - UNION operations combining similar datasets with consistent structure

When you check the refresh history above, you should see `REFRESH_TYPE = 'INCREMENTAL'` for these tables when they process new data, demonstrating their efficiency. The `BYTES_SCANNED` and `CREDITS_USED` metrics will also be lower compared to full refresh operations, showing the performance benefits of incremental processing.


## Query Types Showcase: Complex Incremental Operations

These examples demonstrate more sophisticated query patterns that still support incremental refresh mode. These operations are more complex than basic aggregations but can still efficiently process only changed data:

1. **LATERAL with FLATTEN()** - Parsing nested JSON/VARIANT data using LATERAL FLATTEN to extract array elements
2. **CTEs with Incremental Logic** - Common Table Expressions that build incremental transformations step by step  
3. **Window Functions (Incremental-Safe)** - LAG, LEAD, FIRST_VALUE, LAST_VALUE over partitioned data
4. **Incremental Deduplication** - ROW_NUMBER() OVER (PARTITION BY key ORDER BY timestamp) for latest records

These patterns maintain incremental refresh capability while providing advanced analytical functionality.


In [ ]:
-- Example 1: LATERAL with FLATTEN() Dynamic Table
-- Demonstrates parsing nested JSON/VARIANT data for incremental processing
CREATE OR REPLACE DYNAMIC TABLE json_sales_flattened
    LAG = '1 MINUTE'
    WAREHOUSE = XSMALL_WH
AS
SELECT 
    s.custid,
    f.value:"prodid"::number as product_id,
    f.value:"quantity"::number as quantity,
    f.value:"purchase_amount"::number(10,2) as amount,
    f.value:"purchase_date"::date as purchase_date,
    CURRENT_TIMESTAMP() as processed_timestamp
FROM salesdata s,
LATERAL FLATTEN(input => ARRAY_CONSTRUCT(s.purchase)) f
WHERE f.value IS NOT NULL;


In [ ]:
-- Example 2: CTEs with Incremental Logic Dynamic Table
-- Demonstrates multi-step transformations using Common Table Expressions
CREATE OR REPLACE DYNAMIC TABLE customer_segmentation_cte
    LAG = '1 MINUTE'
    WAREHOUSE = XSMALL_WH
AS
WITH customer_totals AS (
    SELECT 
        customer_id,
        customer_name,
        SUM(saleprice) as total_spent,
        COUNT(*) as transaction_count,
        AVG(saleprice) as avg_transaction
    FROM salesreport
    GROUP BY customer_id, customer_name
),
customer_percentiles AS (
    SELECT 
        customer_id,
        customer_name,
        total_spent,
        transaction_count,
        avg_transaction,
        NTILE(5) OVER (ORDER BY total_spent) as spending_quintile
    FROM customer_totals
)
SELECT 
    customer_id,
    customer_name,
    total_spent,
    transaction_count,
    avg_transaction,
    spending_quintile,
    CASE 
        WHEN spending_quintile = 5 THEN 'VIP'
        WHEN spending_quintile >= 4 THEN 'High Value'
        WHEN spending_quintile >= 3 THEN 'Medium Value'
        ELSE 'Standard'
    END as customer_tier
FROM customer_percentiles;


In [ ]:
-- Example 3: Window Functions (Incremental-Safe) Dynamic Table
-- Demonstrates LAG, LEAD, FIRST_VALUE, LAST_VALUE over partitioned data
CREATE OR REPLACE DYNAMIC TABLE sales_trends_window
    LAG = '1 MINUTE'
    WAREHOUSE = XSMALL_WH
AS
SELECT 
    customer_id,
    customer_name,
    product_id,
    saleprice,
    creationtime,
    -- Previous transaction amount for this customer
    LAG(saleprice, 1) OVER (PARTITION BY customer_id ORDER BY creationtime) as prev_transaction,
    -- Next transaction amount for this customer  
    LEAD(saleprice, 1) OVER (PARTITION BY customer_id ORDER BY creationtime) as next_transaction,
    -- First transaction amount for this customer
    FIRST_VALUE(saleprice) OVER (PARTITION BY customer_id ORDER BY creationtime) as first_transaction,
    -- Most recent transaction amount for this customer
    LAST_VALUE(saleprice) OVER (PARTITION BY customer_id ORDER BY creationtime ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as latest_transaction,
    -- Calculate spending trend
    CASE 
        WHEN LAG(saleprice, 1) OVER (PARTITION BY customer_id ORDER BY creationtime) IS NULL THEN 'First Purchase'
        WHEN saleprice > LAG(saleprice, 1) OVER (PARTITION BY customer_id ORDER BY creationtime) THEN 'Increasing'
        WHEN saleprice < LAG(saleprice, 1) OVER (PARTITION BY customer_id ORDER BY creationtime) THEN 'Decreasing'
        ELSE 'Stable'
    END as spending_trend
FROM salesreport;


In [ ]:
-- Example 4: Incremental Deduplication Dynamic Table
-- Demonstrates ROW_NUMBER() OVER (PARTITION BY key ORDER BY timestamp) for latest records
CREATE OR REPLACE DYNAMIC TABLE latest_customer_transactions
    LAG = '1 MINUTE'
    WAREHOUSE = XSMALL_WH
AS
SELECT 
    customer_id,
    customer_name,
    product_id,
    product_name,
    saleprice,
    quantity,
    creationtime,
    customer_sk
FROM (
    SELECT 
        customer_id,
        customer_name,
        product_id,
        product_name,
        saleprice,
        quantity,
        creationtime,
        customer_sk,
        -- Get the latest transaction for each customer-product combination
        ROW_NUMBER() OVER (PARTITION BY customer_id, product_id ORDER BY creationtime DESC) as rn
    FROM salesreport
) ranked
WHERE rn = 1  -- Only keep the most recent transaction per customer-product pair
QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id, product_id ORDER BY creationtime DESC) = 1;


In [ ]:
-- Verify Complex Incremental Operations Dynamic Tables
SELECT 'JSON_SALES_FLATTENED' as table_name, COUNT(*) as record_count FROM json_sales_flattened
UNION ALL
SELECT 'CUSTOMER_SEGMENTATION_CTE' as table_name, COUNT(*) as record_count FROM customer_segmentation_cte
UNION ALL
SELECT 'SALES_TRENDS_WINDOW' as table_name, COUNT(*) as record_count FROM sales_trends_window
UNION ALL
SELECT 'LATEST_CUSTOMER_TRANSACTIONS' as table_name, COUNT(*) as record_count FROM latest_customer_transactions;


In [ ]:
-- Test Complex Incremental Operations - Insert new data
INSERT INTO salesdata SELECT * FROM table(gen_cust_purchase(3000, 2));

-- Check updated base table count
SELECT 'SALESDATA_COMPLEX_TEST' as table_name, COUNT(*) as record_count FROM salesdata;


In [ ]:
-- Check Complex Incremental Operations refresh history
-- These should show REFRESH_TYPE = 'INCREMENTAL' demonstrating advanced incremental capabilities
SELECT 
    NAME as dynamic_table_name,
    REFRESH_TYPE,
    REFRESH_REASON,
    DATA_TIMESTAMP,
    REFRESH_START_TIME,
    REFRESH_END_TIME,
    CREDITS_USED,
    BYTES_SCANNED,
    ROWS_PRODUCED
FROM 
    TABLE(INFORMATION_SCHEMA.DYNAMIC_TABLE_REFRESH_HISTORY())
WHERE 
    NAME IN ('JSON_SALES_FLATTENED', 'CUSTOMER_SEGMENTATION_CTE', 'SALES_TRENDS_WINDOW', 'LATEST_CUSTOMER_TRANSACTIONS')
    AND REFRESH_START_TIME >= DATEADD('hour', -1, CURRENT_TIMESTAMP())
ORDER BY 
    NAME, DATA_TIMESTAMP DESC, REFRESH_END_TIME DESC;


## Query Types Showcase: Non-Deterministic Functions (Full Refresh Mode)

These examples demonstrate query patterns that require **FULL refresh mode** due to their non-deterministic nature. These operations cannot be incrementally computed because they depend on complete dataset ordering or produce different results with the same input:

1. **ANY_VALUE Aggregations** - Selecting arbitrary values from grouped data where order is not guaranteed
2. **RANK Functions** - RANK() and DENSE_RANK() operations that depend on complete dataset ordering  
3. **ROW_NUMBER Global Ordering** - ROW_NUMBER() without proper partitioning that requires full dataset scan

**Important:** These Dynamic Tables will show `REFRESH_TYPE = 'FULL'` in the refresh history, demonstrating when full refresh is necessary.


In [ ]:
-- Example 1: ANY_VALUE Aggregations Dynamic Table (Full Refresh Mode)
-- Demonstrates arbitrary value selection requiring full dataset knowledge
CREATE OR REPLACE DYNAMIC TABLE any_value_customer_sample
    LAG = '1 MINUTE'
    WAREHOUSE = XSMALL_WH
AS
SELECT 
    DATE(creationtime) as sales_date,
    COUNT(*) as total_transactions,
    -- ANY_VALUE is non-deterministic - could return different values with same input
    ANY_VALUE(customer_name) as sample_customer_name,
    ANY_VALUE(product_name) as sample_product_name,
    ANY_VALUE(saleprice) as sample_transaction_amount,
    -- These functions return arbitrary values from each group
    ANY_VALUE(customer_id) as sample_customer_id,
    ANY_VALUE(product_id) as sample_product_id,
    MAX(saleprice) as max_transaction,
    MIN(saleprice) as min_transaction,
    AVG(saleprice) as avg_transaction
FROM salesreport
GROUP BY DATE(creationtime);


In [ ]:
-- Example 2: RANK Functions Dynamic Table (Full Refresh Mode)  
-- Demonstrates ranking operations requiring complete dataset ordering
CREATE OR REPLACE DYNAMIC TABLE customer_spending_ranks
    LAG = '1 MINUTE'
    WAREHOUSE = XSMALL_WH
AS
SELECT 
    customer_id,
    customer_name,
    total_spent,
    transaction_count,
    -- RANK and DENSE_RANK require full dataset to determine accurate rankings
    RANK() OVER (ORDER BY total_spent DESC) as spending_rank,
    DENSE_RANK() OVER (ORDER BY total_spent DESC) as dense_spending_rank,
    PERCENT_RANK() OVER (ORDER BY total_spent) as spending_percentile,
    -- These rankings can change when new data arrives
    RANK() OVER (ORDER BY transaction_count DESC) as transaction_count_rank,
    DENSE_RANK() OVER (ORDER BY transaction_count DESC) as dense_transaction_rank
FROM (
    SELECT 
        customer_id,
        customer_name,
        SUM(saleprice) as total_spent,
        COUNT(*) as transaction_count
    FROM salesreport
    GROUP BY customer_id, customer_name
) customer_aggregates;


In [ ]:
-- Example 3: ROW_NUMBER Global Ordering Dynamic Table (Full Refresh Mode)
-- Demonstrates global row numbering requiring full dataset scan
CREATE OR REPLACE DYNAMIC TABLE global_transaction_sequence
    LAG = '1 MINUTE'
    WAREHOUSE = XSMALL_WH
AS
SELECT 
    customer_id,
    customer_name,
    product_id,
    product_name,
    saleprice,
    creationtime,
    -- Global ROW_NUMBER without partitioning requires full dataset knowledge
    ROW_NUMBER() OVER (ORDER BY creationtime, customer_id, product_id) as global_transaction_sequence,
    ROW_NUMBER() OVER (ORDER BY saleprice DESC) as high_value_sequence,
    ROW_NUMBER() OVER (ORDER BY saleprice ASC) as low_value_sequence,
    -- These sequences will change when new data is inserted anywhere in the ordered set
    CASE 
        WHEN ROW_NUMBER() OVER (ORDER BY saleprice DESC) <= 100 THEN 'Top 100 Transaction'
        WHEN ROW_NUMBER() OVER (ORDER BY saleprice ASC) <= 100 THEN 'Bottom 100 Transaction'
        ELSE 'Middle Transaction'
    END as transaction_classification
FROM salesreport;


In [ ]:
-- Verify Non-Deterministic Functions Dynamic Tables
SELECT 'ANY_VALUE_CUSTOMER_SAMPLE' as table_name, COUNT(*) as record_count FROM any_value_customer_sample
UNION ALL
SELECT 'CUSTOMER_SPENDING_RANKS' as table_name, COUNT(*) as record_count FROM customer_spending_ranks
UNION ALL
SELECT 'GLOBAL_TRANSACTION_SEQUENCE' as table_name, COUNT(*) as record_count FROM global_transaction_sequence;


In [ ]:
-- Test Non-Deterministic Functions - Insert new data
INSERT INTO salesdata SELECT * FROM table(gen_cust_purchase(2000, 1));

-- Check updated base table count
SELECT 'SALESDATA_NON_DETERMINISTIC_TEST' as table_name, COUNT(*) as record_count FROM salesdata;


In [ ]:
-- Check Non-Deterministic Functions refresh history
-- These should show REFRESH_TYPE = 'FULL' due to non-deterministic query patterns
SELECT 
    NAME as dynamic_table_name,
    REFRESH_TYPE,
    REFRESH_REASON,
    DATA_TIMESTAMP,
    REFRESH_START_TIME,
    REFRESH_END_TIME,
    CREDITS_USED,
    BYTES_SCANNED,
    ROWS_PRODUCED,
    -- Highlight full refresh operations
    CASE 
        WHEN REFRESH_TYPE = 'FULL' THEN '⚠ FULL REFRESH REQUIRED'
        WHEN REFRESH_TYPE = 'INCREMENTAL' THEN '✓ INCREMENTAL REFRESH'
        ELSE '? UNKNOWN TYPE'
    END as refresh_mode_indicator
FROM 
    TABLE(INFORMATION_SCHEMA.DYNAMIC_TABLE_REFRESH_HISTORY())
WHERE 
    NAME IN ('ANY_VALUE_CUSTOMER_SAMPLE', 'CUSTOMER_SPENDING_RANKS', 'GLOBAL_TRANSACTION_SEQUENCE')
    AND REFRESH_START_TIME >= DATEADD('hour', -1, CURRENT_TIMESTAMP())
ORDER BY 
    NAME, DATA_TIMESTAMP DESC, REFRESH_END_TIME DESC;


## Query Types Showcase: Complex Analytics (Full Refresh Mode)

These examples demonstrate advanced analytical functions that require **FULL refresh mode** due to their computational complexity. These operations need complete dataset access to produce accurate results:

1. **Percentile Functions** - PERCENTILE_CONT, PERCENTILE_DISC requiring full dataset analysis
2. **Cross-Partition Analytics** - Window functions that span across all data without proper partitioning
3. **Complex Statistical Functions** - STDDEV, VARIANCE, CORR over entire dataset

**Important:** These Dynamic Tables will show `REFRESH_TYPE = 'FULL'` due to their analytical complexity and need for complete dataset knowledge.


In [ ]:
-- Example 1: Percentile Functions Dynamic Table (Full Refresh Mode)
-- Demonstrates percentile calculations requiring full dataset analysis
CREATE OR REPLACE DYNAMIC TABLE sales_percentile_analysis
    LAG = '1 MINUTE'
    WAREHOUSE = XSMALL_WH
AS
SELECT 
    'SALES_PERCENTILES' as analysis_type,
    COUNT(*) as total_transactions,
    -- Percentile functions require full dataset to calculate accurate percentiles
    PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY saleprice) as q1_sale_price,
    PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY saleprice) as median_sale_price,
    PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY saleprice) as q3_sale_price,
    PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY saleprice) as p90_sale_price,
    PERCENTILE_CONT(0.95) WITHIN GROUP (ORDER BY saleprice) as p95_sale_price,
    PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY saleprice) as p99_sale_price,
    -- Discrete percentiles
    PERCENTILE_DISC(0.50) WITHIN GROUP (ORDER BY saleprice) as discrete_median,
    PERCENTILE_DISC(0.90) WITHIN GROUP (ORDER BY saleprice) as discrete_p90,
    -- Distribution metrics
    MIN(saleprice) as min_price,
    MAX(saleprice) as max_price,
    CURRENT_TIMESTAMP() as analysis_timestamp
FROM salesreport;


In [ ]:
-- Example 2: Cross-Partition Analytics Dynamic Table (Full Refresh Mode)
-- Demonstrates window functions spanning entire dataset without partitioning
CREATE OR REPLACE DYNAMIC TABLE cross_partition_analytics
    LAG = '1 MINUTE'
    WAREHOUSE = XSMALL_WH
AS
SELECT 
    customer_id,
    customer_name,
    product_id,
    saleprice,
    creationtime,
    -- Cross-partition window functions requiring full dataset knowledge
    SUM(saleprice) OVER () as total_sales_across_all_data,
    AVG(saleprice) OVER () as overall_average_sale_price,
    COUNT(*) OVER () as total_transaction_count,
    -- Running totals across entire unpartitioned dataset
    SUM(saleprice) OVER (ORDER BY creationtime ROWS UNBOUNDED PRECEDING) as running_total_all_sales,
    -- Percentage of total sales
    ROUND((saleprice / SUM(saleprice) OVER ()) * 100, 4) as pct_of_total_sales,
    -- Global standard deviation and variance
    STDDEV(saleprice) OVER () as global_price_stddev,
    VARIANCE(saleprice) OVER () as global_price_variance,
    -- Global min/max
    MIN(saleprice) OVER () as global_min_price,
    MAX(saleprice) OVER () as global_max_price
FROM salesreport;


In [ ]:
-- Example 3: Complex Statistical Functions Dynamic Table (Full Refresh Mode)
-- Demonstrates advanced statistical calculations requiring full dataset
CREATE OR REPLACE DYNAMIC TABLE advanced_statistics_analysis
    LAG = '1 MINUTE'
    WAREHOUSE = XSMALL_WH
AS
SELECT 
    'STATISTICAL_ANALYSIS' as analysis_type,
    COUNT(*) as sample_size,
    -- Complex statistical functions requiring complete dataset
    AVG(saleprice) as mean_sale_price,
    STDDEV(saleprice) as price_std_deviation,
    VARIANCE(saleprice) as price_variance,
    STDDEV_POP(saleprice) as price_std_dev_population,
    VAR_POP(saleprice) as price_variance_population,
    -- Correlation analysis (requires multiple columns and full dataset)
    CORR(saleprice, quantity) as price_quantity_correlation,
    -- Advanced statistical measures
    SKEW(saleprice) as price_skewness,
    KURTOSIS(saleprice) as price_kurtosis,
    -- Distribution analysis
    (MAX(saleprice) - MIN(saleprice)) as price_range,
    PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY saleprice) as median_price,
    (PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY saleprice) - 
     PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY saleprice)) as interquartile_range,
    -- Count of outliers (values beyond 2 standard deviations)
    SUM(CASE WHEN ABS(saleprice - AVG(saleprice) OVER()) > 2 * STDDEV(saleprice) OVER() THEN 1 ELSE 0 END) as outlier_count,
    CURRENT_TIMESTAMP() as analysis_timestamp
FROM salesreport;


In [ ]:
-- Verify Complex Analytics Dynamic Tables
SELECT 'SALES_PERCENTILE_ANALYSIS' as table_name, COUNT(*) as record_count FROM sales_percentile_analysis
UNION ALL
SELECT 'CROSS_PARTITION_ANALYTICS' as table_name, COUNT(*) as record_count FROM cross_partition_analytics
UNION ALL
SELECT 'ADVANCED_STATISTICS_ANALYSIS' as table_name, COUNT(*) as record_count FROM advanced_statistics_analysis;


In [ ]:
-- Test Complex Analytics - Insert new data
INSERT INTO salesdata SELECT * FROM table(gen_cust_purchase(2500, 1));

-- Check updated base table count
SELECT 'SALESDATA_COMPLEX_ANALYTICS_TEST' as table_name, COUNT(*) as record_count FROM salesdata;


In [ ]:
-- Check Complex Analytics refresh history
-- These should show REFRESH_TYPE = 'FULL' due to complex analytical requirements
SELECT 
    NAME as dynamic_table_name,
    REFRESH_TYPE,
    REFRESH_REASON,
    DATA_TIMESTAMP,
    REFRESH_START_TIME,
    REFRESH_END_TIME,
    CREDITS_USED,
    BYTES_SCANNED,
    ROWS_PRODUCED,
    -- Highlight the complexity and refresh requirements
    CASE 
        WHEN REFRESH_TYPE = 'FULL' THEN '⚠ FULL REFRESH - Complex Analytics'
        WHEN REFRESH_TYPE = 'INCREMENTAL' THEN '✓ INCREMENTAL REFRESH'
        ELSE '? UNKNOWN TYPE'
    END as analytics_refresh_indicator,
    -- Performance metrics for full refresh operations
    ROUND(CREDITS_USED, 4) as credits_consumed,
    ROUND(BYTES_SCANNED / 1024 / 1024, 2) as mb_scanned
FROM 
    TABLE(INFORMATION_SCHEMA.DYNAMIC_TABLE_REFRESH_HISTORY())
WHERE 
    NAME IN ('SALES_PERCENTILE_ANALYSIS', 'CROSS_PARTITION_ANALYTICS', 'ADVANCED_STATISTICS_ANALYSIS')
    AND REFRESH_START_TIME >= DATEADD('hour', -1, CURRENT_TIMESTAMP())
ORDER BY 
    NAME, DATA_TIMESTAMP DESC, REFRESH_END_TIME DESC;


In [ ]:
-- Check Dynamic Tables after refresh (wait a minute for automatic refresh)
SELECT COUNT(*) as salesreport_count FROM salesreport;

## Dynamic Tables with Python UDTFs

We'll create a Python UDTF for cumulative sum calculations and use it in a Dynamic Table to demonstrate how to combine Python functions with Dynamic Tables for advanced analytics.


In [ ]:
-- Create Python UDTF for cumulative sum calculation
USE SCHEMA DEMO.DT_DEMO;

CREATE OR REPLACE FUNCTION sum_table (INPUT_NUMBER number)
  RETURNS TABLE (running_total number)
  LANGUAGE PYTHON
  RUNTIME_VERSION = '3.10'
  HANDLER = 'gen_sum_table'
AS
$$
# Define handler class
class gen_sum_table :

  ## Define __init__ method to initialize the variable
  def __init__(self) :
    self._running_sum = 0
  
  ## Define process method
  def process(self, input_number: float) :
    # Increment running sum with data from the input row
    new_total = self._running_sum + input_number
    self._running_sum = new_total

    yield(new_total,)
$$;


In [ ]:
-- Create Dynamic Table using Python UDTF for cumulative purchase totals
CREATE OR REPLACE DYNAMIC TABLE cumulative_purchase
    LAG = '1 MINUTE'
    WAREHOUSE=XSMALL_WH
AS
    SELECT 
        MONTH(creationtime) monthNum,
        YEAR(creationtime) yearNum,
        customer_id, 
        saleprice,
        running_total 
    FROM 
        salesreport,
        TABLE(sum_table(saleprice) OVER (PARTITION BY creationtime,customer_id ORDER BY creationtime, customer_id));


In [ ]:
-- Verify cumulative purchase calculations
SELECT * FROM cumulative_purchase LIMIT 10;


## Data Validation and Alerting with Dynamic Tables

Create a Dynamic Table for inventory monitoring that identifies products with low stock levels and set up automated alerts.


In [ ]:
-- Create Dynamic Table for product inventory alerts
USE SCHEMA DEMO.DT_DEMO;

CREATE OR REPLACE DYNAMIC TABLE PROD_INV_ALERT
    LAG = '1 MINUTE'
    WAREHOUSE=XSMALL_WH
AS
    SELECT 
        S.PRODUCT_ID, 
        S.PRODUCT_NAME,
        CREATIONTIME AS LATEST_SALES_DATE,
        STOCK AS BEGINING_STOCK,
        SUM(S.QUANTITY) OVER (PARTITION BY S.PRODUCT_ID ORDER BY CREATIONTIME) TOTALUNITSOLD, 
        (STOCK - TOTALUNITSOLD) AS UNITSLEFT,
        ROUND(((STOCK-TOTALUNITSOLD)/STOCK) *100,2) PERCENT_UNITLEFT,
        CURRENT_TIMESTAMP() AS ROWCREATIONTIME
    FROM SALESREPORT S JOIN PROD_STOCK_INV ON PRODUCT_ID = PID
    QUALIFY ROW_NUMBER() OVER (PARTITION BY PRODUCT_ID ORDER BY CREATIONTIME DESC) = 1;


In [ ]:
-- Add new sales data to test incremental refresh behavior
INSERT INTO salesdata SELECT * FROM table(gen_cust_purchase(5000,1));

-- Wait for Dynamic Tables to refresh (they have 1 MINUTE lag), then validate incremental processing
-- This query should be run after ~1-2 minutes to allow Dynamic Tables to refresh

-- IMPORTANT: Run this query after waiting 1-2 minutes for the Dynamic Tables to refresh
SELECT 
    'INCREMENTAL_REFRESH_VALIDATION' as analysis_type,
    NAME as dynamic_table_name,
    REFRESH_MODE as actual_refresh_mode,
    REFRESH_MODE_REASON as mode_explanation,
    DATEDIFF('seconds', REFRESH_START_TIME, REFRESH_END_TIME) as refresh_duration_seconds,
    ROWS_PRODUCED as rows_affected_by_refresh,
    BYTES_SCANNED as bytes_processed,
    CREDITS_USED as warehouse_credits,
    -- Key indicator: For incremental refresh, ROWS_PRODUCED should be much less than total table size
    CASE 
        WHEN REFRESH_MODE = 'INCREMENTAL' AND ROWS_PRODUCED < 1000 
        THEN '✓ CONFIRMED INCREMENTAL' 
        WHEN REFRESH_MODE = 'INCREMENTAL' AND ROWS_PRODUCED >= 1000 
        THEN '⚠ INCREMENTAL BUT HIGH ROW COUNT'
        WHEN REFRESH_MODE = 'FULL' 
        THEN '✗ FULL REFRESH OCCURRED'
        ELSE '? UNKNOWN STATE'
    END as incremental_validation_status
FROM 
    TABLE(INFORMATION_SCHEMA.DYNAMIC_TABLE_REFRESH_HISTORY())
WHERE 
    NAME IN ('SALES_AGGREGATIONS','RECENT_HIGH_VALUE_SALES','CUSTOMER_PRODUCT_ANALYSIS','UNIFIED_TRANSACTION_FEED')
    AND REFRESH_START_TIME > DATEADD('minute', -5, CURRENT_TIMESTAMP())  -- Only recent refreshes
    AND DATA_TIMESTAMP IS NOT NULL
ORDER BY 
    REFRESH_START_TIME DESC;


In [ ]:
-- Check for products with low inventory (less than 10%)
SELECT * FROM prod_inv_alert WHERE percent_unitleft < 10;


In [ ]:
-- Create notification integration and alert for low inventory
-- Note: Update email address before running
CREATE OR REPLACE NOTIFICATION INTEGRATION -- IF NOT EXISTS
    notification_emailer
    TYPE=EMAIL
    ENABLED=TRUE
    ALLOWED_RECIPIENTS=('aaron.gavic@snowflake.com')  -- UPDATE THIS EMAIL
    COMMENT = 'email integration to update on low product inventory levels';

CREATE OR REPLACE ALERT alert_low_inv
  WAREHOUSE = XSMALL_WH
  SCHEDULE = '30 MINUTE'
  IF (EXISTS (
      SELECT *
      FROM prod_inv_alert
      WHERE percent_unitleft < 10 AND ROWCREATIONTIME > SNOWFLAKE.ALERT.LAST_SUCCESSFUL_SCHEDULED_TIME()
  ))
  THEN CALL SYSTEM$SEND_EMAIL(
                'notification_emailer', -- notification integration to use
                'aaron.gavic@snowflake.com', -- Email (UPDATE THIS)
                'Email Alert: Low Inventory of products', -- Subject
                'Inventory running low for certain products. Please check the inventory report in Snowflake table prod_inv_alert' -- Body of email
);

-- Alerts are paused by default, so resume it
ALTER ALERT alert_low_inv RESUME;


In [ ]:
-- Add more sales data to potentially trigger alerts
INSERT INTO salesdata SELECT * FROM table(gen_cust_purchase(10000,2));


## Monitoring and Management

Dynamic Tables provide comprehensive monitoring capabilities to track refresh history, performance, and troubleshoot issues.


In [ ]:
-- Monitor Dynamic Tables refresh history
SELECT * 
FROM 
    TABLE(INFORMATION_SCHEMA.DYNAMIC_TABLE_REFRESH_HISTORY())
WHERE 
    NAME IN ('SALESREPORT','CUSTOMER_SALES_DATA_HISTORY','PROD_INV_ALERT','CUMULATIVE_PURCHASE')
ORDER BY 
    DATA_TIMESTAMP DESC, REFRESH_END_TIME DESC 
LIMIT 10;


In [ ]:
-- Show Dynamic Tables information and refresh modes
SHOW DYNAMIC TABLES;


In [ ]:
-- Monitor alerts
SHOW ALERTS;

In [ ]:
SELECT *
FROM
  TABLE(INFORMATION_SCHEMA.ALERT_HISTORY(
    SCHEDULED_TIME_RANGE_START
      =>DATEADD('hour',-1,CURRENT_TIMESTAMP())))
WHERE
    NAME = 'ALERT_LOW_INV'
ORDER BY SCHEDULED_TIME DESC;


## Dynamic Table Operations

Dynamic Tables can be suspended, resumed, and manually refreshed as needed for operational control.


In [ ]:
-- Suspend Dynamic Tables (stops automatic refresh)
ALTER DYNAMIC TABLE customer_sales_data_history SUSPEND;
ALTER DYNAMIC TABLE salesreport SUSPEND;
ALTER DYNAMIC TABLE prod_inv_alert SUSPEND;
ALTER DYNAMIC TABLE cumulative_purchase SUSPEND;


In [ ]:
-- Resume Dynamic Tables (restarts automatic refresh)
ALTER DYNAMIC TABLE customer_sales_data_history RESUME;
ALTER DYNAMIC TABLE salesreport RESUME;
ALTER DYNAMIC TABLE prod_inv_alert RESUME;
ALTER DYNAMIC TABLE cumulative_purchase RESUME;


In [ ]:
-- Manually refresh a Dynamic Table
ALTER DYNAMIC TABLE salesreport REFRESH;


## Cleanup

**Important:** Run this section to prevent ongoing warehouse costs and credit consumption.


In [ ]:
-- Suspend all Dynamic Tables to stop refresh cycles
ALTER DYNAMIC TABLE customer_sales_data_history SUSPEND;
ALTER DYNAMIC TABLE salesreport SUSPEND;
ALTER DYNAMIC TABLE prod_inv_alert SUSPEND;
ALTER DYNAMIC TABLE cumulative_purchase SUSPEND;

-- Suspend Basic Incremental Operations Dynamic Tables
ALTER DYNAMIC TABLE daily_sales_summary SUSPEND;
ALTER DYNAMIC TABLE high_value_recent_sales SUSPEND;
ALTER DYNAMIC TABLE customer_purchase_analysis SUSPEND;
ALTER DYNAMIC TABLE unified_transaction_log SUSPEND;

-- Suspend Complex Incremental Operations Dynamic Tables
ALTER DYNAMIC TABLE json_sales_flattened SUSPEND;
ALTER DYNAMIC TABLE customer_segmentation_cte SUSPEND;
ALTER DYNAMIC TABLE sales_trends_window SUSPEND;
ALTER DYNAMIC TABLE latest_customer_transactions SUSPEND;

-- Suspend Non-Deterministic Functions Dynamic Tables
ALTER DYNAMIC TABLE any_value_customer_sample SUSPEND;
ALTER DYNAMIC TABLE customer_spending_ranks SUSPEND;
ALTER DYNAMIC TABLE global_transaction_sequence SUSPEND;

-- Suspend Complex Analytics Dynamic Tables
ALTER DYNAMIC TABLE sales_percentile_analysis SUSPEND;
ALTER DYNAMIC TABLE cross_partition_analytics SUSPEND;
ALTER DYNAMIC TABLE advanced_statistics_analysis SUSPEND;

-- Suspend alert to stop consuming warehouse credits
ALTER ALERT alert_low_inv SUSPEND;


In [ ]:
-- Add new sales data to test incremental refresh behavior
INSERT INTO salesdata SELECT * FROM table(gen_cust_purchase(5000,1));

-- Wait for Dynamic Tables to refresh (they have 1 MINUTE lag), then validate incremental processing
-- This query should be run after ~1-2 minutes to allow Dynamic Tables to refresh

-- IMPORTANT: Run this query after waiting 1-2 minutes for the Dynamic Tables to refresh
SELECT 
    'INCREMENTAL_REFRESH_VALIDATION' as analysis_type,
    NAME as dynamic_table_name,
    REFRESH_MODE as actual_refresh_mode,
    REFRESH_MODE_REASON as mode_explanation,
    DATEDIFF('seconds', REFRESH_START_TIME, REFRESH_END_TIME) as refresh_duration_seconds,
    ROWS_PRODUCED as rows_affected_by_refresh,
    BYTES_SCANNED as bytes_processed,
    CREDITS_USED as warehouse_credits,
    -- Key indicator: For incremental refresh, ROWS_PRODUCED should be much less than total table size
    CASE 
        WHEN REFRESH_MODE = 'INCREMENTAL' AND ROWS_PRODUCED < 1000 
        THEN '✓ CONFIRMED INCREMENTAL' 
        WHEN REFRESH_MODE = 'INCREMENTAL' AND ROWS_PRODUCED >= 1000 
        THEN '⚠ INCREMENTAL BUT HIGH ROW COUNT'
        WHEN REFRESH_MODE = 'FULL' 
        THEN '✗ FULL REFRESH OCCURRED'
        ELSE '? UNKNOWN STATE'
    END as incremental_validation_status
FROM 
    TABLE(INFORMATION_SCHEMA.DYNAMIC_TABLE_REFRESH_HISTORY())
WHERE 
    NAME IN ('SALES_AGGREGATIONS','RECENT_HIGH_VALUE_SALES','CUSTOMER_PRODUCT_ANALYSIS','UNIFIED_TRANSACTION_FEED')
    AND REFRESH_START_TIME > DATEADD('minute', -5, CURRENT_TIMESTAMP())  -- Only recent refreshes
    AND DATA_TIMESTAMP IS NOT NULL
ORDER BY 
    REFRESH_START_TIME DESC;


In [ ]:
-- Add new sales data to test incremental refresh behavior
INSERT INTO salesdata SELECT * FROM table(gen_cust_purchase(5000,1));

-- Wait for Dynamic Tables to refresh (they have 1 MINUTE lag), then validate incremental processing
-- This query should be run after ~1-2 minutes to allow Dynamic Tables to refresh

-- IMPORTANT: Run this query after waiting 1-2 minutes for the Dynamic Tables to refresh
SELECT 
    'INCREMENTAL_REFRESH_VALIDATION' as analysis_type,
    NAME as dynamic_table_name,
    REFRESH_MODE as actual_refresh_mode,
    REFRESH_MODE_REASON as mode_explanation,
    DATEDIFF('seconds', REFRESH_START_TIME, REFRESH_END_TIME) as refresh_duration_seconds,
    ROWS_PRODUCED as rows_affected_by_refresh,
    BYTES_SCANNED as bytes_processed,
    CREDITS_USED as warehouse_credits,
    -- Key indicator: For incremental refresh, ROWS_PRODUCED should be much less than total table size
    CASE 
        WHEN REFRESH_MODE = 'INCREMENTAL' AND ROWS_PRODUCED < 1000 
        THEN '✓ CONFIRMED INCREMENTAL' 
        WHEN REFRESH_MODE = 'INCREMENTAL' AND ROWS_PRODUCED >= 1000 
        THEN '⚠ INCREMENTAL BUT HIGH ROW COUNT'
        WHEN REFRESH_MODE = 'FULL' 
        THEN '✗ FULL REFRESH OCCURRED'
        ELSE '? UNKNOWN STATE'
    END as incremental_validation_status
FROM 
    TABLE(INFORMATION_SCHEMA.DYNAMIC_TABLE_REFRESH_HISTORY())
WHERE 
    NAME IN ('SALES_AGGREGATIONS','RECENT_HIGH_VALUE_SALES','CUSTOMER_PRODUCT_ANALYSIS','UNIFIED_TRANSACTION_FEED')
    AND REFRESH_START_TIME > DATEADD('minute', -5, CURRENT_TIMESTAMP())  -- Only recent refreshes
    AND DATA_TIMESTAMP IS NOT NULL
ORDER BY 
    REFRESH_START_TIME DESC;


In [ ]:
-- Optional: Complete cleanup by dropping all demo objects
-- Uncomment the lines below if you want to remove all demo objects

 DROP SCHEMA DEMO.DT_DEMO CASCADE;
 DROP DATABASE DEMO;


## Summary

This notebook demonstrated the key capabilities of Snowflake Dynamic Tables:

- **Declarative Pipeline Creation:** Define data transformations with simple SQL queries
- **Automatic Orchestration:** Snowflake manages refresh scheduling based on target lag
- **Incremental Processing:** Efficient updates that process only changed data
- **Python Integration:** Use UDTFs for advanced analytics within Dynamic Tables
- **Data Validation:** Build automated data quality checks and alerts
- **Monitoring:** Comprehensive observability through built-in functions and Snowsight
- **Operational Control:** Suspend, resume, and manually refresh as needed

Dynamic Tables simplify data engineering by providing a declarative approach to building continuous data pipelines without the complexity of traditional orchestration tools.
